# Slipskip

Let's preprocess the weather data first. Only Helsinki (Kumpula) for now. 

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
df_hel = pd.read_csv("./Helsinki_weather.csv")
df_oul = pd.read_csv("./Oulu-weather.csv")
df_kuo = pd.read_csv("./Kuopio-weather.csv")
df_jyv = pd.read_csv("./Jyväskylä-weather.csv")
df_lah = pd.read_csv("./Lahti-weather.csv")

df_hel["City"] = "HEL"
df_oul["City"] = "OUL"
df_kuo["City"] = "KUO"
df_jyv["City"] = "JYV"
df_lah["City"] = "LAH"

df_intro = pd.concat([df_hel, df_oul, df_kuo, df_jyv, df_lah])

print(df_intro)

      Vuosi  Kk  Pv    Klo Aikavyöhyke Sademäärä (mm) Lumensyvyys (cm)  \
0      2013  11  24  00:00         UTC             -1               -1   
1      2013  11  24  06:00         UTC              -                -   
2      2013  11  25  00:00         UTC              0               -1   
3      2013  11  25  06:00         UTC              -                -   
4      2013  11  26  00:00         UTC              1                -   
...     ...  ..  ..    ...         ...            ...              ...   
2538   2022   9  20  06:00         UTC              -                -   
2539   2022   9  21  00:00         UTC             -1               -1   
2540   2022   9  21  06:00         UTC              -                -   
2541   2022   9  22  00:00         UTC             -1               -1   
2542   2022   9  22  06:00         UTC              -                -   

     Ilman lämpötila (degC) Maanpintaminimi (degC) Ylin lämpötila (degC)  \
0                       0.7        

In [3]:
df_intro["date"] = pd.to_datetime(dict(year=df_intro.Vuosi, month=df_intro.Kk, day=df_intro.Pv))
df = df_intro.drop(columns=["Vuosi", "Kk", "Pv", "Aikavyöhyke"])#, "Unnamed: 0"])
df = df.rename(columns={"Sademäärä (mm)": "rainfall_mm", "Lumensyvyys (cm)": "snow_depth_cm", "Ilman lämpötila (degC)": "air_temp", 
    "Ylin lämpötila (degC)": "max_temp", "Alin lämpötila (degC)": "min_temp", "Maanpintaminimi (degC)": "min_ground_temp", "City" : "City"})
cols = ['date', 'rainfall_mm', 'snow_depth_cm', 'air_temp', 'max_temp', 'min_temp', 'min_ground_temp', 'Klo', 'City']
df = df[cols]

print(df)

           date rainfall_mm snow_depth_cm air_temp max_temp min_temp  \
0    2013-11-24          -1            -1      0.7      4.4     -1.4   
1    2013-11-24           -             -        -        -        -   
2    2013-11-25           0            -1     -1.7        0     -2.8   
3    2013-11-25           -             -        -        -        -   
4    2013-11-26           1             -       -2      2.8     -6.6   
...         ...         ...           ...      ...      ...      ...   
2538 2022-09-20           -             -        -        -        -   
2539 2022-09-21          -1            -1      8.2      9.9        7   
2540 2022-09-21           -             -        -        -        -   
2541 2022-09-22          -1            -1        7     11.4      1.9   
2542 2022-09-22           -             -        -        -        -   

     min_ground_temp    Klo City  
0                  -  00:00  HEL  
1                  0  06:00  HEL  
2                  -  00:00  H

In [4]:
df2 = df
maanpintaminimi = df[["date", "min_ground_temp", "Klo"]]

df2 = df2.drop(["min_ground_temp"], axis=1)
df2 = df2.where(df["Klo"] == "00:00")
df2 = df2.dropna()

maanpintaminimi = maanpintaminimi.where(df["Klo"] == "06:00")
maanpintaminimi = maanpintaminimi.dropna()
print(maanpintaminimi)
print(maanpintaminimi.shape)

print("*"*70)

df2 = df2.drop(["Klo"], axis=1)
print(df2)
maanpintaminimi = maanpintaminimi.drop(["Klo"], axis=1)
print(maanpintaminimi)

merged = df2.merge(right=maanpintaminimi, how="left")
print("+"*70)
print(merged)

           date min_ground_temp    Klo
1    2013-11-24               0  06:00
3    2013-11-25            -8.6  06:00
5    2013-11-26           -10.8  06:00
7    2013-11-27             0.8  06:00
9    2013-11-28             2.5  06:00
...         ...             ...    ...
2534 2022-09-18             8.5  06:00
2536 2022-09-19             8.1  06:00
2538 2022-09-20             8.7  06:00
2540 2022-09-21             6.5  06:00
2542 2022-09-22            -1.4  06:00

[6091 rows x 3 columns]
(6091, 3)
**********************************************************************
           date rainfall_mm snow_depth_cm air_temp max_temp min_temp City
0    2013-11-24          -1            -1      0.7      4.4     -1.4  HEL
2    2013-11-25           0            -1     -1.7        0     -2.8  HEL
4    2013-11-26           1             -       -2      2.8     -6.6  HEL
6    2013-11-27          -1            -1      4.4      5.4      2.8  HEL
8    2013-11-28           0            -1      3.5      

In [8]:
# merged["Maanpintaminimi (degC)"] = merged["Maanpintaminimi (degC)"].interpolate()
merged2 = merged.mask(merged == "-", np.nan)

print(merged2.isna().any())
print("-"*70)

merged2 = merged2
merged2[["rainfall_mm", "snow_depth_cm", "air_temp", "max_temp", "min_temp", "min_ground_temp"]] = merged2[["rainfall_mm", "snow_depth_cm", "air_temp", "max_temp", "min_temp", "min_ground_temp"]].astype('float64')
print(merged2.dtypes)
merged2 = merged2[["rainfall_mm", "snow_depth_cm", "air_temp", "max_temp", "min_temp", "min_ground_temp"]].interpolate(axis=0)
print(merged2.isna().any())
print("+"*70)

merged2 = pd.concat([merged[["date","City"]],merged2], axis=1)
print(merged2)
print("*"*70)
print(merged2.isna().any())

# merged2.to_csv("./wether_data_trimmed_helsinki_kumpula.csv")
merged2.to_csv("./wether_data_trimmed_Finland.csv")

date               False
rainfall_mm         True
snow_depth_cm       True
air_temp            True
max_temp            True
min_temp            True
City               False
min_ground_temp     True
dtype: bool
----------------------------------------------------------------------
date               datetime64[ns]
rainfall_mm               float64
snow_depth_cm             float64
air_temp                  float64
max_temp                  float64
min_temp                  float64
City                       object
min_ground_temp           float64
dtype: object
rainfall_mm        False
snow_depth_cm      False
air_temp           False
max_temp           False
min_temp           False
min_ground_temp    False
dtype: bool
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            date City  rainfall_mm  snow_depth_cm  air_temp  max_temp  \
0     2013-11-24  HEL         -1.0           -1.0       0.7       4.4   
1     2013-11-25  HEL          0.0           -1.0   

In [9]:
merge3 = merged2
print(merge3)
print("*"*70)
# merge2["Sademäärä (mm)"] = merge2["Sademäärä (mm)"].astype(float)
# merge2["Lumensyvyys (cm)"] = merge2["Lumensyvyys (cm)"].astype(float)
merge3["rainfall_mm"] = merge3["rainfall_mm"].where(merge3["rainfall_mm"] >= 0, 0)
merge3["snow_depth_cm"] = merge3["snow_depth_cm"].mask(merge3["snow_depth_cm"] == -1, 0)
print(merge3.head(30))

merge3.to_csv("./wether_data_trimmed_Finland_no_neg.csv")
# merge3.to_csv("./wether_data_trimmed_helsinki_kumpula_without_negative_rain_or_snow.csv")

            date City  rainfall_mm  snow_depth_cm  air_temp  max_temp  \
0     2013-11-24  HEL         -1.0           -1.0       0.7       4.4   
1     2013-11-25  HEL          0.0           -1.0      -1.7       0.0   
2     2013-11-26  HEL          1.0           -1.0      -2.0       2.8   
3     2013-11-27  HEL         -1.0           -1.0       4.4       5.4   
4     2013-11-28  HEL          0.0           -1.0       3.5       6.3   
...          ...  ...          ...            ...       ...       ...   
27888 2022-09-21  LAH         -1.0           -1.0       8.2       9.9   
27889 2022-09-21  LAH         -1.0           -1.0       8.2       9.9   
27890 2022-09-22  LAH         -1.0           -1.0       7.0      11.4   
27891 2022-09-22  LAH         -1.0           -1.0       7.0      11.4   
27892 2022-09-22  LAH         -1.0           -1.0       7.0      11.4   

       min_temp  min_ground_temp  
0          -1.4              0.0  
1          -2.8             -8.6  
2          -6.6   